## College Football Machine Learning 2008-2013

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV 

In [2]:
# Load the data
from google.colab import files
uploaded = files.upload()

file_name='goldcopyhalfstats.csv'
#df = pd.read_csv(file_name, skiprows=0)[:-2]
df=pd.read_csv(file_name)

Saving goldcopyhalfstats.csv to goldcopyhalfstats.csv


In [3]:
df.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,0,14,5,Akron ...,Mid-American Conference ...,1,28,BTEAM,17,24,84,84,129,129,40,40,154,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,0,3,8,Alabama ...,Southeastern Conference ...,1,20,BTEAM,9,27,45,45,123,123,56,56,116,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,0,13,8,Alabama ...,Southeastern Conference ...,1,42,BTEAM,16,15,90,90,157,157,129,129,139,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,1,38,27,Appalachian State ...,Southern ...,0,0,ATEAM,25,16,136,136,91,91,140,140,71,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,1,21,37,Auburn ...,Southeastern Conference ...,0,21,TIED,16,27,61,61,157,157,165,165,125,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942


# Add Home Team and Perform Basic Data Cleaning

In [4]:
# The game code (usually) has both teams codes in it at the beginning.  The home team is the team whose code is SECOND in the game code
# HOWEVER...every now and then there is a game that is played on a neutral field where one of the teams does not have its code in the game code
# Two examples are at indices 49, 72, and 1952
# We need to account for these instances 

# Create an empty list to hold the home teams
# Since the stats for a team mean "different" if the a Team is at home
# and vice-verse, we need separate columns for each team to designate
# if they are home
# i.e. how would one variable tell you which set of other variables
# it was linked to?
aHome=[]
bHome=[]

# The string comparison is...kinda weird for some reason so I will compare
# strings to a known 'neutral game string'
neutral=str(df.iloc[1]['site'])

for index,row in df.iterrows():
  gameCode=str(row['game_code'])
  aTeam=str(row['a_team_code'])
  bTeam=str(row['b_team_code'])
  site=str(row['site'])

  homeTeam=''
  try:
    i1=gameCode.index(aTeam)
  except ValueError:  # For when a team code isn't in game code
    homeTeam='NOT IN CODE'
  try:
    i2=gameCode.index(bTeam)
  except ValueError:  # For when a team code isn't in game code
    homeTeam='NOT IN CODE'

  if(homeTeam==''):
      if(site==neutral):
        aHome.append(0)  # Neutral Field Game
        bHome.append(0)
      elif (i1>i2):      
        aHome.append(1)  # The aTeam team code was second  
        bHome.append(0)   
      else:
        aHome.append(0)  
        bHome.append(1)  # The aTeam team code was second
  else:
      # One of the teams did not have its team code in the game code
      if(index < 150):
        print(index)
      aHome.append(0)  # Neutral Field Game
      bHome.append(0)

df['a_home']=aHome
df['b_home']=bHome

49
72
75
80
103
105
106
108
117
147
148


In [5]:
df.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,a_home,b_home
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,0,14,5,Akron ...,Mid-American Conference ...,1,28,BTEAM,17,24,84,84,129,129,40,40,154,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,1,0
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,0,3,8,Alabama ...,Southeastern Conference ...,1,20,BTEAM,9,27,45,45,123,123,56,56,116,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992,0,0
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,0,13,8,Alabama ...,Southeastern Conference ...,1,42,BTEAM,16,15,90,90,157,157,129,129,139,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929,1,0
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,1,38,27,Appalachian State ...,Southern ...,0,0,ATEAM,25,16,136,136,91,91,140,140,71,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965,1,0
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,1,21,37,Auburn ...,Southeastern Conference ...,0,21,TIED,16,27,61,61,157,157,165,165,125,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1,0


In [6]:
df.iloc[72]

year                                                                 2008
game_code                                                 147074620131102
a_team_name             Wake Forest                                   ...
a_name_conference       Atlantic Coast Conference                     ...
a_team_code                                                           749
a_result                                                                1
a_points                                                                3
b_team_code                                                           147
team_name               Clemson                                       ...
name_conference         Atlantic Coast Conference                     ...
b_result                                                                0
b_points                                                                0
half_leader                                                    ATEAM     
a_rush                                

In [7]:
# Drop the null columns where all values are null
df2 = df.dropna(axis='columns', how='all').copy()

# Drop the null rows
df2 = df2.dropna()

df2.reset_index(inplace=True, drop=True)

print(len(df2))
df2.head(10)

4789


,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,a_home,b_home
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,0,14,5,Akron ...,Mid-American Conference ...,1,28,BTEAM,17,24,84,84,129,129,40,40,154,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,1,0
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,0,3,8,Alabama ...,Southeastern Conference ...,1,20,BTEAM,9,27,45,45,123,123,56,56,116,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992,0,0
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,0,13,8,Alabama ...,Southeastern Conference ...,1,42,BTEAM,16,15,90,90,157,157,129,129,139,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929,1,0
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,1,38,27,Appalachian State ...,Southern ...,0,0,ATEAM,25,16,136,136,91,91,140,140,71,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965,1,0
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,1,21,37,Auburn ...,Southeastern Conference ...,0,21,TIED,16,27,61,61,157,157,165,165,125,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1,0
5,2013,37023420140106,Florida State ...,Atlantic Coast Conference ...,234,1,10,37,Auburn ...,Southeastern Conference ...,0,21,BTEAM,18,29,137,137,131,131,62,62,107,107,15,13,10,6,2,3,20,15,-11,11,1/6/2014,15,Rose Bowl ...,NEUTRAL,Pasadena ...,CA,91136,Grass,1922,0,0
6,2012,43074220120908,Virginia Tech ...,Atlantic Coast Conference ...,742,1,21,43,Austin Peay ...,OVC ...,0,0,ATEAM,17,22,-47,-47,64,64,151,151,11,11,14,11,5,7,2,3,10,20,21,21,9/8/2012,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965,1,0
7,2009,51074920090905,Wake Forest ...,Atlantic Coast Conference ...,749,0,7,51,Baylor ...,Big 12 Conference ...,1,10,BTEAM,21,14,68,68,43,43,44,44,107,107,6,16,1,4,5,2,40,10,-3,3,9/5/2009,83,Groves Stadium ...,TEAM,Winston Salem ...,NC,31500,FieldTurf,1968,1,0
8,2013,61023420130921,Florida State ...,Atlantic Coast Conference ...,234,1,33,61,Bethune-Cookman ...,Mid-Eastern ...,0,0,ATEAM,20,26,46,46,91,91,109,109,61,61,16,10,9,6,4,1,37,16,33,33,9/21/2013,33,Doak Campbell Stadium ...,TEAM,Tallahassee ...,FL,80000,Grass,1950,1,0
9,2011,61041520111001,Miami (Florida) ...,Atlantic Coast Conference ...,415,1,14,61,Bethune-Cookman ...,Mid-Eastern ...,0,7,ATEAM,9,24,-22,-22,-6,-6,107,107,89,89,9,20,2,9,3,6,28,40,7,7,10/1/2011,30,Sun Life Stadium ...,TEAM,Miami ...,FL,75235,Grass,1987,1,0


In [8]:
df3=df2.copy()
df3["result"]=df3['a_result']
df3=df3.drop(['a_result', 'b_result'], axis=1)

df3=df3.drop(['a_defense_rush_yards','b_defense_rush_yards','a_defense_pass_yards','b_defense_pass_yards'],axis=1)

df3.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,a_home,b_home,result
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,14,5,Akron ...,Mid-American Conference ...,28,BTEAM,17,24,84,129,40,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,1,0,0
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,3,8,Alabama ...,Southeastern Conference ...,20,BTEAM,9,27,45,123,56,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992,0,0,0
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,13,8,Alabama ...,Southeastern Conference ...,42,BTEAM,16,15,90,157,129,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929,1,0,0
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,38,27,Appalachian State ...,Southern ...,0,ATEAM,25,16,136,91,140,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965,1,0,1
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,21,37,Auburn ...,Southeastern Conference ...,21,TIED,16,27,61,157,165,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1,0,1


#Filter Data

In [9]:
df_filter=df3.copy()

# Throw out "blowout" games
blowoutThresh=14
df_filter=df_filter[df_filter['half_diff_score_abs'] <= blowoutThresh]

# reset index
df_filter.reset_index(inplace=True, drop=True)

print(len(df_filter))
df_filter.head(5)

3242


,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,a_home,b_home,result
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,14,5,Akron ...,Mid-American Conference ...,28,BTEAM,17,24,84,129,40,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,1,0,0
1,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,21,37,Auburn ...,Southeastern Conference ...,21,TIED,16,27,61,157,165,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1,0,1
2,2013,37023420140106,Florida State ...,Atlantic Coast Conference ...,234,10,37,Auburn ...,Southeastern Conference ...,21,BTEAM,18,29,137,131,62,107,15,13,10,6,2,3,20,15,-11,11,1/6/2014,15,Rose Bowl ...,NEUTRAL,Pasadena ...,CA,91136,Grass,1922,0,0,1
3,2009,51074920090905,Wake Forest ...,Atlantic Coast Conference ...,749,7,51,Baylor ...,Big 12 Conference ...,10,BTEAM,21,14,68,43,44,107,6,16,1,4,5,2,40,10,-3,3,9/5/2009,83,Groves Stadium ...,TEAM,Winston Salem ...,NC,31500,FieldTurf,1968,1,0,0
4,2011,61041520111001,Miami (Florida) ...,Atlantic Coast Conference ...,415,14,61,Bethune-Cookman ...,Mid-Eastern ...,7,ATEAM,9,24,-22,-6,107,89,9,20,2,9,3,6,28,40,7,7,10/1/2011,30,Sun Life Stadium ...,TEAM,Miami ...,FL,75235,Grass,1987,1,0,1


In [10]:
df_filter.columns

Index(['year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
       'a_points', 'b_team_code', 'team_name', 'name_conference', 'b_points',
       'half_leader', 'a_rush', 'b_rush', 'a_offense_rush_yards',
       'b_offense_rush_yards', 'a_offense_pass_yards', 'b_offense_pass_yards',
       'a_pass', 'b_pass', 'a_incomplete_pass', 'b_incomplete_pass',
       'a_penalty', 'b_penalty', 'a_penalty_yards', 'b_penalty_yards',
       'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
       'stadium_name', 'site', 'city', 'state', 'capacity', 'surface',
       'year_opened', 'a_home', 'b_home', 'result'],
      dtype='object')

# Split the Data into Training and Testing and Scale

In [11]:
# Create our features
X = df_filter.copy()
#X=X.drop({'year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
#          'b_team_code', 'team_name', 'name_conference','half_leader',
#          'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
#          'stadium_name', 'site', 'city', 'state', 'capacity',
#          'year_opened', 'result'},axis=1)

X=X.drop({'year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
          'b_team_code', 'team_name', 'name_conference','half_leader',
          'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
          'stadium_name', 'site', 'city', 'state', 'capacity',
          'year_opened', 'result'},axis=1)

# Need to convert columns with strings to numeric
# Get column names that are non-numeric
column_names=X.select_dtypes(exclude=[np.number]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for columnName in column_names:
  X[columnName]=le.fit_transform(X[columnName])

# Create our target
y = df_filter['result'].copy()

X.head()

,a_points,b_points,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,surface,a_home,b_home
0,14,28,17,24,84,129,40,154,10,15,3,3,2,0,10,0,3,1,0
1,21,21,16,27,61,157,165,125,24,15,8,7,4,2,35,20,4,1,0
2,10,21,18,29,137,131,62,107,15,13,10,6,2,3,20,15,4,0,0
3,7,10,21,14,68,43,44,107,6,16,1,4,5,2,40,10,3,1,0
4,14,7,9,24,-22,-6,107,89,9,20,2,9,3,6,28,40,4,1,0


In [12]:
# Check the balance of our target values
print(len(X))
abs(X['a_points']-X['b_points']).value_counts()

# We have a lot of games with a 7-point spread at half compared to the others (makes sense)
# May want to look at stratifying?  So that our model predicts well at any spread

3242


7     516
3     397
14    375
10    343
0     330
4     319
11    234
1     145
6     138
13    137
8     106
2      59
9      58
5      52
12     33
dtype: int64

# Sampling Methods and Scale


In [15]:

from sklearn.model_selection import train_test_split

# stratify by point spread
stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.25)

# Scale the Data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#scaler = StandardScaler()
#scaler=MinMaxScaler()
scaler=StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#X_train_scaled = X_train
#X_test_scaled = X_test


In [16]:
abs(X_train['a_points']-X_train['b_points']).value_counts()

7     387
3     298
14    281
10    257
0     247
4     239
11    175
1     109
6     103
13    103
8      80
9      44
2      44
5      39
12     25
dtype: int64

# Models

## Import Dependencies

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


## Logistic Regression

In [18]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=1500,
                                random_state=42)

result=classifier.fit(X_train_scaled, y_train)

In [19]:
ml_df=pd.DataFrame(columns=X.columns.tolist())

log_coefs=list(zip(X.columns.tolist(),
                   result.coef_.tolist()[0]))
log_coefs.sort(key = lambda x:abs(x[1]),reverse=True)
print(log_coefs)

[('b_points', -1.1552783243634375), ('a_points', 1.1204075379395277), ('a_offense_pass_yards', 0.19064817621656419), ('b_home', -0.15546142133046742), ('a_pass', -0.13111116146431062), ('a_penalty_yards', -0.12743229430454386), ('a_rush', -0.1026879958876395), ('a_home', 0.09930369299306654), ('a_offense_rush_yards', 0.09418385772555028), ('a_incomplete_pass', 0.0884729611977676), ('a_penalty', 0.08682170804905873), ('b_penalty_yards', 0.08042070802958064), ('b_offense_rush_yards', -0.07440578842729077), ('b_incomplete_pass', 0.06529750490039898), ('b_pass', -0.059017525150585934), ('b_penalty', 0.041445478774979276), ('b_offense_pass_yards', -0.02791284069399231), ('surface', 0.025585160065513857), ('b_rush', 0.013944061467267801)]


In [20]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
print(classifier.get_params())
results.head(10)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


,Prediction,Actual
0,0,0
1,1,1
2,0,1
3,1,0
4,1,1
5,1,1
6,1,1
7,1,0
8,1,0
9,1,1


In [21]:
logRegAcc=balanced_accuracy_score(y_test, y_pred)
noDroplogRegAcc=logRegAcc
print('balanced accuracy score = ',logRegAcc)

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual B", "Actual A"], columns=["Predicted B", "Predicted A"])

print("")
print(cm_df)

# Print the imbalanced classification report
print("")
print(classification_report_imbalanced(y_test, y_pred))

balanced accuracy score =  0.6562957406323184

          Predicted B  Predicted A
Actual B          165          219
Actual A           50          377

                   pre       rec       spe        f1       geo       iba       sup

          0       0.77      0.43      0.88      0.55      0.62      0.36       384
          1       0.63      0.88      0.43      0.74      0.62      0.40       427

avg / total       0.70      0.67      0.64      0.65      0.62      0.38       811



# Sensitivity Tests

In [22]:
# We will drop each column and see what the balanced score gives us
X.columns
print('No Drop Balanced Accuracy = ',round(noDroplogRegAcc,2))
for column in X.columns:
  #Creata new Dataframe with dropped column
  Xtest=X.drop(column,axis=1).copy()

  # Gotta train, test, split using same random seed
  # stratify by point spread
  stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
  X_train, X_test, y_train, y_test = train_test_split(Xtest, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.25)

  # Scale the Data
  #scaler = StandardScaler()
  #scaler=MinMaxScaler()
  scaler=StandardScaler()
  X_scaler = scaler.fit(X_train)

  X_train_scaled = X_scaler.transform(X_train)
  X_test_scaled = X_scaler.transform(X_test)

  # ML with logistic Regression
  classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2500,
                                random_state=42)

  result=classifier.fit(X_train_scaled, y_train)
  y_pred = classifier.predict(X_test)
  logResAcc=balanced_accuracy_score(y_test, y_pred)

  print('Dropped ',column,', balanced acc. = ',round(logResAcc,2))


No Drop Balanced Accuracy =  0.66
Dropped  a_points , balanced acc. =  0.5
Dropped  b_points , balanced acc. =  0.5
Dropped  a_rush , balanced acc. =  0.57
Dropped  b_rush , balanced acc. =  0.65
Dropped  a_offense_rush_yards , balanced acc. =  0.67
Dropped  b_offense_rush_yards , balanced acc. =  0.6
Dropped  a_offense_pass_yards , balanced acc. =  0.64
Dropped  b_offense_pass_yards , balanced acc. =  0.67
Dropped  a_pass , balanced acc. =  0.66
Dropped  b_pass , balanced acc. =  0.6
Dropped  a_incomplete_pass , balanced acc. =  0.58
Dropped  b_incomplete_pass , balanced acc. =  0.55
Dropped  a_penalty , balanced acc. =  0.71
Dropped  b_penalty , balanced acc. =  0.69
Dropped  a_penalty_yards , balanced acc. =  0.62
Dropped  b_penalty_yards , balanced acc. =  0.64
Dropped  surface , balanced acc. =  0.54
Dropped  a_home , balanced acc. =  0.62
Dropped  b_home , balanced acc. =  0.59


# Drop unneeded features

In [25]:
Xnew=X.drop({'a_penalty','b_penalty',
             'a_home'
            },axis=1).copy()


# Gotta train, test, split using same random seed
# stratify by point spread
stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
X_train, X_test, y_train, y_test = train_test_split(Xnew, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.25)

# Scale the Data
#scaler = StandardScaler()
#scaler=MinMaxScaler()
scaler=StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# ML with logistic Regression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2500,
                                random_state=42)

result=classifier.fit(X_train_scaled, y_train)
y_pred = classifier.predict(X_test)
logResAcc=balanced_accuracy_score(y_test, y_pred)
print('Old Balanced Accuracy = ',round(noDroplogRegAcc,2))
print('New Balanced Accuracy = ',round(logResAcc,2))

Old Balanced Accuracy =  0.66
New Balanced Accuracy =  0.65


In [26]:
Xnew.columns

Index(['a_points', 'b_points', 'a_rush', 'b_rush', 'a_offense_rush_yards',
       'b_offense_rush_yards', 'a_offense_pass_yards', 'b_offense_pass_yards',
       'a_pass', 'b_pass', 'a_incomplete_pass', 'b_incomplete_pass',
       'a_penalty_yards', 'b_penalty_yards', 'surface', 'b_home'],
      dtype='object')

In [27]:
coefficients=result.coef_.tolist()[0]
coefficients

[1.1556664108577066,
 -1.218726384106107,
 -0.15838663289394267,
 0.0010705342614478885,
 0.15498772317260845,
 -0.062359939756317706,
 0.0941987518122639,
 0.005082518973450786,
 -0.04395006593427325,
 -0.0902058863617246,
 0.08204983687970466,
 0.05273256858848149,
 -0.02604424893181787,
 0.11175416948656264,
 -0.02613643103770641,
 -0.22695470699675338]

# Create Prediction Function!

In [28]:
# Create Function to predict future values
# Let's give it some default values
# Also re-order the input parameters
def get_winner(a_points, b_points, a_offense_rush_yards, \
          b_offense_rush_yards, a_incomplete_pass, \
          b_incomplete_pass, a_penalty, b_penalty, a_penalty_yards, \
          b_penalty_yards, surface, b_home):
       r1 = a_points, b_points, a_offense_rush_yards, \
          b_offense_rush_yards, a_incomplete_pass, \
          b_incomplete_pass, a_penalty, b_penalty, a_penalty_yards, \
          b_penalty_yards, surface, b_home
       return classifier.predict([r1])[0]

In [29]:
from math import exp
def get_winner2(a_points, b_points, a_offense_rush_yards, \
          b_offense_rush_yards, a_incomplete_pass, \
          b_incomplete_pass, a_penalty, b_penalty, a_penalty_yards, \
          b_penalty_yards, surface, b_home):
  
      r1 = a_points, b_points, a_offense_rush_yards, \
              b_offense_rush_yards, a_incomplete_pass, \
              b_incomplete_pass, a_penalty, b_penalty, a_penalty_yards, \
              b_penalty_yards, surface, b_home
      coefs = [1.1433200927084646,-1.2078283033984454,0.08515619138009485, \
                -0.05672591750867272,0.09213321441084038,0.013490550175894282, \
                0.02592144799616056,-0.07482047209659458,-0.095450375002064,
                0.19728405912508,0.03912180890023299,-0.21778627453337315]
      index=0;
      linearSum=0;
      for num in r1:
        linearSum = linearSum + coefs[index]*num
        index = index + 1
      sigmoid=int(round(1/(1+exp(-1*linearSum)),0))
      return sigmoid